modified from Song's `hsc_um2` repo

In [1]:
import numpy as np
from numpy import linalg
from __future__ import division, print_function, unicode_literals

import time
import argparse

import emcee
print(emcee.__version__) #need version 3 (pip install emcee==3.0rc1)
import yaml

from scipy.stats import gaussian_kde
from astropy.table import Table, Column
from astropy.cosmology import FlatLambdaCDM


3.0rc1


## parse config file 

In [8]:
def parse_config(config_file):
    """Prepare configurations.
    Read configuration parameters from an input .yaml file.
    """
    cfg = yaml.load(open(config_file))

    return cfg
config_initial = parse_config('mcmc_default_config.yaml')
print(config_initial)

{'um_min_scatter': 0.01, 'param_ini': [0.599, 3.669, -0.048, 0.02, 0.8, 0.11], 'obs_smf_cov': 's16a_wide2_massive_smf_m100_m10_cov.npy', 'obs_wl_sample': 's16a_wide2_massive_boxbin3_default', 'mcmc_moves_burnin': 'walk', 'param_upp': [1.0, 8.0, 0.0, 0.2, 1.0, 0.3], 'obs_smf_full_fits': 'primus_smf_z0.3_0.4.fits', 'mcmc_nwalkers_burnin': 128, 'um_mtot_nbin_min': 7, 'param_sig': [0.05, 0.1, 0.02, 0.005, 0.05, 0.05], 'mcmc_wl_only': False, 'mcmc_nsamples': 200, 'um_model': 'um_smdpl_0.7124_new_vagc_mpeak_11.5.npy', 'obs_smf_inn': 's16a_wide2_massive_smf_m10_new.npy', 'obs_h0': 0.7, 'obs_minn_col': 'logm_10', 'um_wl_add_stellar': False, 'obs_dir': '../data/s16a_massive_wide2', 'obs_area': 145.0, 'um_redshift': 0.3637, 'mcmc_wl_weight': 1.0, 'um_wl_nbin': 22, 'obs_cat': 's16a_wide2_massive_fsps1_imgsub_use_short.fits', 'mcmc_prefix': 'asap_smdpl', 'mcmc_smf_only': False, 'mcmc_nthreads': 2, 'um_dir': '../data/s16a_massive_wide2/um2', 'um_min_mvir': 11.5, 'obs_omega_m': 0.307, 'um_lbox': 400

## define global variables for config and data files

In [ ]:
def load_observed_data(cfg, verbose=True):
    """Load the observed data."""
    # Galaxy catalog.
    obs_mass = Table.read(os.path.join(cfg['obs_dir'],
                                       cfg['obs_cat']))

    obs_minn = np.array(obs_mass[cfg['obs_minn_col']])
    obs_mtot = np.array(obs_mass[cfg['obs_mtot_col']])

    with open(cfg['obs_wl_out'], 'rb') as f:
        # BUG: Tricky work around for pickling Python 2 array in Python 3
        # https://stackoverflow.com/questions/11305790/pickle-incompatibility-of-numpy-arrays-between-python-2-and-3
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        obs_wl_bin, obs_wl_dsigma = u.load()

    cfg['obs_wl_n_bin'] = len(obs_wl_bin)
    if verbose:
        if cfg['obs_wl_n_bin'] > 1:
            print("# There are %d weak lensing profiles in this sample" %
                  cfg['obs_wl_n_bin'])
        else:
            print("# There is 1 weak lensing profile in this sample")

    cfg['obs_dsigma_n_data'] = len(obs_wl_dsigma[0].r) * cfg['obs_wl_n_bin']

    obs_smf_inn = Table.read(cfg['smf_inn_file'])
    obs_smf_tot = Table.read(cfg['smf_tot_file'])

    cfg['obs_smf_inn_min'] = np.min(obs_smf_inn['logm_0'])
    cfg['obs_smf_inn_max'] = np.max(obs_smf_inn['logm_1'])
    cfg['obs_smf_inn_nbin'] = len(obs_smf_inn)

    cfg['obs_smf_tot_min'] = np.min(obs_smf_tot['logm_0'])
    cfg['obs_smf_tot_max'] = np.max(obs_smf_tot['logm_1'])
    cfg['obs_smf_tot_nbin'] = len(obs_smf_tot)

    cfg['obs_ngal_use'] = ((obs_mtot >= cfg['obs_smf_tot_min']) &
                           (obs_minn >= cfg['obs_smf_inn_min'])).sum()

    # TODO : test this margin
    cfg['obs_min_mtot'] = cfg['obs_smf_tot_min'] - 0.1

    cfg['obs_smf_n_data'] = cfg['obs_smf_tot_nbin'] + cfg['obs_smf_inn_nbin']

    obs_smf_cov = np.load(cfg['smf_cov_file'])
    assert cfg['obs_smf_n_data'] == len(obs_smf_cov)
#     else:
#         obs_smf_cov = None

    if verbose:
        print("# SMF for total stellar mass: ")
        print("  %7.4f -- %7.4f in %d bins" % (cfg['obs_smf_tot_min'],
                                               cfg['obs_smf_tot_max'],
                                               cfg['obs_smf_tot_nbin']))
        print("# SMF for inner stellar mass: ")
        print("  %7.4f -- %7.4f in %d bins" % (cfg['obs_smf_inn_min'],
                                               cfg['obs_smf_inn_max'],
                                               cfg['obs_smf_inn_nbin']))

    obs_logms_inn = obs_minn[obs_mtot >= cfg['obs_smf_tot_min']]
    obs_logms_tot = obs_mtot[obs_mtot >= cfg['obs_smf_tot_min']]


    smf_full = Table.read(cfg['obs_smf_full_file'])
    smf_full[smf_full['smf'] <= 0]['smf'] = 1E-8
    smf_full[smf_full['smf_low'] <= 0]['smf_low'] = 1E-9
    smf_full[smf_full['smf_upp'] <= 0]['smf_upp'] = 1E-7
    obs_smf_full = smf_full
    if verbose:
        print("# Pre-computed full SMF: %s" % cfg['obs_smf_full_fits'])

    if verbose:
        print("# For inner stellar mass: ")
        print("    %d bins at %5.2f < logMinn < %5.2f" %
              (cfg['obs_smf_inn_nbin'], cfg['obs_smf_inn_min'],
               cfg['obs_smf_inn_max']))
        print("# For total stellar mass: ")
        print("    %d bins at %5.2f < logMtot < %5.2f" %
              (cfg['obs_smf_tot_nbin'], cfg['obs_smf_tot_min'],
               cfg['obs_smf_tot_max']))

    obs_zmin = np.nanmin(obs_mass[cfg['obs_z_col']])
    obs_zmax = np.nanmax(obs_mass[cfg['obs_z_col']])

    obs_volume = ((cfg['obs_cosmo'].comoving_volume(obs_zmax) -
                   cfg['obs_cosmo'].comoving_volume(obs_zmin)) *
                  (cfg['obs_area'] / 41254.0)).value
    cfg['obs_volume'] = obs_volume

    if verbose:
        print("# The volume of the HSC data is %15.2f Mpc^3" % obs_volume)

    return {'obs_mass': obs_mass,
            'obs_minn': obs_minn, 'obs_mtot': obs_mtot,
            'obs_logms_inn': obs_logms_inn, 'obs_logms_tot': obs_logms_tot,
            'obs_wl_bin': obs_wl_bin, 'obs_wl_dsigma': obs_wl_dsigma,
            'obs_smf_inn': obs_smf_inn, 'obs_smf_tot': obs_smf_tot,
            'obs_smf_full': obs_smf_full, 'obs_smf_cov': obs_smf_cov,
            'obs_volume': obs_volume}, cfg


def config_observed_data(cfg, verbose=True):
    """Config parameters for observed data."""
    # This is for HSC observation

    cfg['obs_cosmo'] = FlatLambdaCDM(H0=cfg['obs_h0'] * 100,
                                     Om0=cfg['obs_omega_m'])
    
    # --------------------------------------------------- #

    # -------------- Observed Data Related -------------- #
    # Catalog for observed galaxies
    if verbose:
        print("# Stellar mass catalog: %s" % cfg['obs_cat'])

    # --------------------------------------------------- #
    # Observed weak lensing delta sigma profiles
    if verbose:
        print("# Weak lensing profile sample: %s" % cfg['obs_wl_sample'])

    obs_wl_dir = os.path.join(cfg['obs_dir'], 'dsigma')
    cfg['obs_wl_out'] = os.path.join(
        obs_wl_dir, cfg['obs_wl_sample'] + '_dsigma_results.pkl')

    # --------------------------------------------------- #
    # Observed stellar mass functions
    cfg['smf_inn_file'] = os.path.join(cfg['obs_dir'], 'smf', cfg['obs_smf_inn'])
    if verbose:
        print("# Pre-computed SMF for inner logMs: %s" % cfg['smf_inn_file'])

    cfg['smf_tot_file'] = os.path.join(cfg['obs_dir'], 'smf',
    if verbose:
        print("# Pre-computed SMF for total logMs: %s" % cfg['smf_tot_file'])

    cfg['smf_cov_file'] = os.path.join(cfg['obs_dir'], 'smf', cfg['obs_smf_cov'])
#     else:
#         cfg['smf_cov_file'] = None
    if verbose:
        print("# Covariances for SMFs: %s" % cfg['smf_cov_file'])

    # Total stellar mass function for comparison (optional)
    cfg['obs_smf_full_file'] = os.path.join(cfg['obs_dir'], cfg['obs_smf_full_fits'])

    if verbose:
        print('# Using %s as inner stellar mass.' %
              cfg['obs_minn_col'])
        print('# Using %s as total stellar mass.' %
              cfg['obs_mtot_col'])

    return cfg


def load_um_data(cfg):
    """Load the UniverseMachine data."""
    um_mock = Table(np.load(os.path.join(cfg['um_dir'],
                                         cfg['um_model'])))

    # Only select the useful columns
    cols_use = ['halo_id', 'upid', 'sm', 'icl', 'x', 'y', 'z',
                'mtot_galaxy', 'mstar_mhalo', 'logms_gal',
                'logms_icl', 'logms_tot', 'logms_halo',
                'logmh_vir', 'logmh_peak', 'logmh_host']
    um_mock_use = um_mock[cols_use]

    # Value added a few useful columns
    um_mock_use.add_column(Column(data=(um_mock_use['mtot_galaxy'] /
                                        um_mock_use['mstar_mhalo']),
                                  name='frac_cen_tot'))
    um_mock_use.add_column(Column(data=(um_mock_use['sm'] /
                                        um_mock_use['mtot_galaxy']),
                                  name='frac_ins_cen'))
    um_mock_use.add_column(Column(data=(um_mock_use['icl'] /
                                        um_mock_use['mtot_galaxy']),
                                  name='frac_exs_cen'))
    um_mock_use = um_mock_use.as_array()

    # Load the pre-compute lensing pairs
    um_mass_encl = np.load(os.path.join(cfg['um_dir'],
                                        cfg['um_wl_cat']))
    assert len(um_mock_use) == len(um_mass_encl)

    # Mask for central galaxies
    mask_central = (um_mock_use['upid'] == -1)

    # Mask for massive enough halo
    mask_mass = (um_mock_use[cfg['um_halo_col']] >= cfg['um_min_mvir'])

    return {'um_mock': um_mock_use[mask_mass],
            'um_mass_encl': um_mass_encl[mask_mass, :],
            'mask_central': mask_central[mask_mass]}


def config_um_data(cfg, verbose=False):
    """Config the UniverseMachine data."""
    # ---------- UniverseMachine Mock Related ----------- #
    cfg['um_cosmo'] = FlatLambdaCDM(H0=cfg['um_h0'] * 100.0,
                                    Om0=cfg['um_omega_m'])

    cfg['um_volume'] = np.power(cfg['um_lbox'] / cfg['um_h0'], 3)
    if verbose:
        print("# The volume of the UniverseMachine mock is %15.2f Mpc^3" %
              cfg['um_volume'])

    return cfg

In [ ]:
def initial_model(config, verbose=True):
    """Initialize the A.S.A.P model."""
    # Configuration for HSC data
    config_obs = config_observed_data(config, verbose=verbose)
    obs_data_use, config_obs = load_observed_data(config_obs, verbose=verbose)

    # Configuration for UniverseMachine data.
    config_obs_um = config_um_data(config_obs, verbose=verbose)
    um_data_use = load_um_data(config_obs_um)

    config_all = setup_model(config_obs_um, verbose=verbose)

    return config_all, obs_data_use, um_data_use

global config, obs_data, um_data

# Load the data
config, obs_data, um_data = initial_model(config_initial)

## probability functions

In [10]:
def flat_prior(param_tuple, param_low, param_upp):
    """Priors of parameters. Return -inf if all parameters are not within bounds."""
    if not np.all([low <= param <= upp for param, low, upp in
                   zip(list(param_tuple), param_low, param_upp)]):
        return -np.inf

    return 0.0

def ln_like(param_tuple, cfg, obs_data, um_data, chi2=False,
                 sep_return=False):
    """Calculate the lnLikelihood of the model."""
    # Unpack the input parameters
    parameters = list(param_tuple)

    # Generate the model predictions
    (um_smf_tot, um_smf_inn, um_dsigma_profs) = predict_model(
            parameters, cfg, obs_data, um_data)       

    # Likelihood for SMFs.
    smf_lnlike = smf_lnlike(
        obs_data['obs_smf_tot'], um_smf_tot,
        obs_data['obs_smf_inn'], um_smf_inn,
        obs_smf_cov=obs_data['obs_smf_cov'])

    # Likelihood for DeltaSigma
    dsigma_lnlike = np.array([
        dsigma_lnlike(obs_dsigma_prof, um_dsigma_prof)
        for (obs_dsigma_prof, um_dsigma_prof) in
        zip(obs_data['obs_wl_dsigma'], um_dsigma_profs)]).sum()

    if not np.isfinite(dsigma_lnlike):
        return -np.inf

    return smf_lnlike + cfg['mcmc_wl_weight'] * dsigma_lnlike

def smf_lnlike(obs_smf_tot, um_smf_tot, obs_smf_inn, um_smf_inn,
                    obs_smf_cov=None):
    """Calculate the likelihood for SMF."""
    smf_mtot_dif = (np.array(um_smf_tot) -
                    np.array(obs_smf_tot['smf']))
    smf_minn_dif = (np.array(um_smf_inn) -
                    np.array(obs_smf_inn['smf']))

    if obs_smf_cov is not None:
        smf_cov_inv = linalg.inv(obs_smf_cov)
        lnlike_norm = -0.5 * ((np.log(2.0 * np.pi) * len(obs_smf_cov)) +
                              np.log(linalg.det(obs_smf_cov)))
        smf_dif = np.concatenate([smf_mtot_dif, smf_minn_dif])

        smf_chi2 = np.dot(smf_dif, np.dot(smf_cov_inv, smf_dif))

        return -0.5 * smf_chi2 + lnlike_norm

    smf_mtot_var = np.array(obs_smf_tot['smf_err'] ** 2)
    smf_minn_var = np.array(obs_smf_inn['smf_err'] ** 2)

    smf_mtot_chi2 = (smf_mtot_dif ** 2 / smf_mtot_var).sum()
    smf_minn_chi2 = (smf_minn_dif ** 2 / smf_minn_var).sum()

    smf_mtot_lnlike = -0.5 * (
        smf_mtot_chi2 + np.log(2 * np.pi * smf_mtot_var).sum())
    smf_minn_lnlike = -0.5 * (
        smf_minn_chi2 + np.log(2 * np.pi * smf_minn_var).sum())

    # print("SMF Tot lnlike/chi2: %f,%f" % (smf_mtot_lnlike,
    #                                       smf_mtot_chi2))
    # print("SMF Inn lnlike/chi2: %f,%f" % (smf_minn_lnlike,
    #                                       smf_minn_chi2))

    return smf_mtot_lnlike + smf_minn_lnlike

def dsigma_lnlike(obs_dsigma_prof, dsigma_um):
    """Calculate the likelihood for WL profile."""
    dsigma_obs = obs_dsigma_prof.sig
    dsigma_obs_err = obs_dsigma_prof.err_s

    dsigma_var = (dsigma_obs_err[:-2] ** 2)
    dsigma_dif = (dsigma_obs[:-2] - dsigma_um[:-2]) ** 2

    dsigma_chi2 = (dsigma_dif / dsigma_var).sum()
    dsigma_lnlike = -0.5 * (dsigma_chi2 +
                            np.log(2 * np.pi * dsigma_var).sum())
    # print("DSigma likelihood / chi2: %f, %f" % (dsigma_lnlike, dsigma_chi2))

    return dsigma_lnlike

def ln_prob_global(param_tuple):
    """Probability function to sample in an MCMC.

    Parameters
    ----------
    param_tuple: tuple of model parameters.

    """
    lp = flat_prior(param_tuple, config['param_low'], config['param_upp'])

    if not np.isfinite(lp):
        return -np.inf

    return lp + ln_like(param_tuple, config, obs_data, um_data)

## setup model

In [11]:
def predict_model(param, config, obs_data, um_data,
                       constant_bin=False, return_all=False,
                       show_smf=False, show_dsigma=False):
    """Return all model predictions.
    Parameters
    ----------
    param: list, array, or tuple.
        Input model parameters.
    cfg : dict
        Configurations of the data and model.
    obs_data: dict
        Dictionary for observed data.
    um_data: dict
        Dictionary for UniverseMachine data.
    constant_bin : boolen
        Whether to use constant bin size for logMs_tot or not.
    return_all : bool, optional
        Return all model information.
    show_smf : bool, optional
        Show the comparison of SMF.
    show_dsigma : bool, optional
        Show the comparisons of WL.
    """
    # Predict stellar mass
    if config['model_type'] == 'simple' or config['model_type'] == 'frac1':
        (logms_mod_inn, logms_mod_tot_all,
         logms_mod_halo_all, mask_mtot) = asap_predict_mass(
             param, config, obs_data, um_data, constant_bin=constant_bin)
        logms_mod_tot = logms_mod_tot_all[mask_mtot]
    else:
        (logms_mod_inn_all, logms_mod_tot_all,
         logms_mod_halo_all, mask_mtot) = asap_predict_mass(
             param, config, obs_data, um_data, constant_bin=constant_bin)
        logms_mod_inn = logms_mod_inn_all[mask_mtot]
        logms_mod_tot = logms_mod_tot_all[mask_mtot]

    # Predict SMFs
    um_smf_tot, um_smf_inn = asap_predict_smf(
        logms_mod_tot, logms_mod_inn, config)

    # Predict DeltaSigma profiles
    um_dsigma_profs = asap_predict_dsigma(
        cfg, obs_data, um_data['um_mock'], um_data['um_mass_encl'],
        logms_mod_tot, logms_mod_inn, mask=mask_mtot,
        add_stellar=config['um_wl_add_stellar'])

#     if show_smf:
#         um_smf_tot_all = get_smf_bootstrap(logms_mod_tot_all,
#                                            cfg['um_volume'],
#                                            20, cfg['obs_min_mtot'], 12.5,
#                                            n_boots=1)
#         _ = plot_mtot_minn_smf(
#             obs_data['obs_smf_tot'], obs_data['obs_smf_inn'],
#             obs_data['obs_mtot'], obs_data['obs_minn'],
#             um_smf_tot, um_smf_inn,
#             logms_mod_tot, logms_mod_inn,
#             obs_smf_full=obs_data['obs_smf_full'],
#             um_smf_tot_all=um_smf_tot_all,
#             not_table=True)

#     if show_dsigma:
#         um_mhalo_tuple = asap_predict_mhalo(
#             obs_data['obs_wl_dsigma'], um_data['um_mock'][mask_mtot],
#             logms_mod_tot, logms_mod_inn)
#         _ = plot_dsigma_profiles(obs_data['obs_wl_dsigma'],
#                                  um_dsigma_profs, obs_mhalo=None,
#                                  um_mhalo=um_mhalo_tuple)

#     if return_all:
#         if cfg['model_type'] == 'simple' or cfg['model_type'] == 'frac1':
#             return (um_smf_tot, um_smf_inn, um_dsigma_profs,
#                     logms_mod_inn, logms_mod_tot_all,
#                     logms_mod_halo_all, mask_mtot)

#         return (um_smf_tot, um_smf_inn, um_dsigma_profs,
#                 logms_mod_inn_all, logms_mod_tot_all,
#                 logms_mod_halo_all, mask_mtot)

    return um_smf_tot, um_smf_inn, um_dsigma_profs

## setup mcmc functions 

In [ ]:
def mcmc_initial_guess(param_initial, param_sigma, n_walkers, n_dims):
    """Initialize guesses for the MCMC run. One guess for each dimension (model parameter) per walker,
    with a small sigma deviation from param_initial. """
    mcmc_position = np.zeros([n_walkers, n_dims])

    for ii, param_0 in enumerate(param_initial):
        mcmc_position[:, ii] = (
            param_0 + param_sigma[ii] * np.random.randn(n_walkers))

    return mcmc_position

def mcmc_setup_moves(config, move_col):
    """Choose the Move object for emcee."""
    if config[move_col] == 'snooker':
        emcee_moves = emcee.moves.DESnookerMove()
    elif config[move_col] == 'stretch':
        emcee_moves = emcee.moves.StretchMove(a=config['mcmc_stretch_a'])
    elif config[move_col] == 'walk':
        emcee_moves = emcee.moves.WalkMove(s=config['mcmc_walk_s'])
    elif config[move_col] == 'kde':
        emcee_moves = emcee.moves.KDEMove()
    elif config[move_col] == 'de':
        emcee_moves = emcee.moves.DEMove(config['mcmc_de_sigma'])
    else:
        raise Exception("Wrong option: stretch, walk, kde, de, snooker")
    
    return emcee_moves

def mcmc_burnin(mcmc_sampler, mcmc_position, config, verbose=True):
    """Run the MCMC chain."""
    # Burn-in
    if verbose:
        print("# Phase: Burn-in ...")
    mcmc_burnin_result = mcmc_sampler.run_mcmc(
        mcmc_position, config['mcmc_nburnin'],
        progress=True)

    mcmc_save_results(mcmc_burnin_result, mcmc_sampler,
                      config['mcmc_burnin_file'], config['mcmc_ndims'],
                      verbose=True)

    # Rest the chains
    mcmc_sampler.reset()

## run fit

In [12]:
#skip paralleization for now
def emcee_fit(config, verbose=True):
    # Initialize the model
    mcmc_ini_position = mcmc_initial_guess(
        config['param_ini'], config['param_sig'], config['mcmc_nwalkers'],
        config['mcmc_ndims'])



    # define the ensemble moves object for walkers during burn in
    burnin_move = mcmc_setup_moves(config, 'mcmc_moves_burnin')
    # define sampler
    burnin_sampler = emcee.EnsembleSampler(
        config['mcmc_nwalkers_burnin'],
        config['mcmc_ndims'],
        ln_prob_global,
        moves=burnin_move) #,pool=pool)

    # Burn-in
    mcmc_burnin_pos, mcmc_burnin_lnp, mcmc_burnin_state = emcee_burnin(
        burnin_sampler, mcmc_ini_position, config, verbose=True)

    # Estimate the Kernel density distributions of final burn-in positions
    # Resample the distributions to get starting positions of the actual run
    mcmc_kde = gaussian_kde(np.transpose(mcmc_burnin_pos), 
                        bw_method='silverman')
    mcmc_new_pos = np.transpose(mcmc_kde.resample(config['mcmc_nwalkers']))

    mcmc_new_ini = (mcmc_new_pos, mcmc_burnin_lnp, mcmc_burnin_state)

    
    # MCMC run
    
    # Decide the Ensemble moves for walkers during burnin
    emcee_move = mcmc_setup_moves(config, 'mcmc_moves')
    # define sampler
    mcmc_sampler = emcee.EnsembleSampler(config['mcmc_nwalkers'],
                                         config['mcmc_ndims'],
                                         ln_prob_global,
                                         moves=emcee_move)

    mcmc_run_result = emcee_run(mcmc_sampler, mcmc_new_ini, config, verbose=True)

    return mcmc_run_result

In [15]:
emcee_fit(config_initial)

# Phase: Burn-in ...
emcee: Exception while calling your likelihood function:
  params: [ 0.60988287  3.47401753 -0.03886581  0.01789452  0.79411887  0.15731723]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/fardila/anaconda/envs/dwarf_lensing/lib/python2.7/site-packages/emcee/ensemble.py", line 488, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-10-f5604446ad1c>", line 100, in ln_prob_global
    return lp + ln_like(param_tuple, config, obs_data, um_data)
NameError: global name 'obs_data' is not defined


NameError: global name 'obs_data' is not defined